In [115]:
import numpy as np
import copy
import math 
import os
import pandas as pd
from scipy.stats import norm
import matplotlib as mpl
xi_1 = 0.98
xi_2 = 0.01
eta_1 = 0.98
eta_2 = 0.01
Q_j = 25
Q_i = 5e6
Q_in = 5e6
epsilon_1 = 0.08
epsilon_2 = 0.7
Q_shed = 1000
V_b = 1e3
V_b_shed = 1e3
C_hum = 0.01
ninfected = 1
NN = 1e-9
p=0.04
MortalityH = 1
T=175

from scipy.stats import triang
def beta_dwp():
    min_val = 1.62
    mode_val = 2.79
    max_val = 4.95    
    c = (mode_val - min_val) / (max_val - min_val)
    dist = triang(c, loc=min_val, scale=max_val - min_val)
    beta_dwp_value = dist.rvs()
    return beta_dwp_value
beta_dwp=beta_dwp()
def beta_awp():
    min_val = 0.0063
    mode_val = 0.06
    max_val = 0.09
    c = (mode_val - min_val) / (max_val - min_val)
    dist = triang(c, loc=min_val, scale=max_val - min_val)
    beta_awp_value = dist.rvs()
    return beta_awp_value
beta_awp=beta_awp()
def beta_fbp():
    min_val = 0.15
    mode_val = 0.28
    max_val = 0.95/2
    c = (mode_val - min_val) / (max_val - min_val)
    dist = triang(c, loc=min_val, scale=max_val - min_val)
    beta_fbp_value = dist.rvs()
    return beta_fbp_value
beta_fbp=beta_fbp()
def beta_fwp():
    min_val = 0.0001
    mode_val = 0.001
    max_val = 0.015
    c = (mode_val - min_val) / (max_val - min_val)
    dist = triang(c, loc=min_val, scale=max_val - min_val)
    beta_fwp_value = dist.rvs()
    return beta_fwp_value
beta_fwp=beta_fwp()
omega_v=np.random.uniform(1e2,4.96e6)
V_b_shed = np.random.uniform(1e2,10**3.2)

In [116]:
# Define the pen below:         
class Pen:
    def __init__(self, npigs, ninfected, location, size, pen_id):
        self.first_day_Clinical = None 
        self.first_day_Carrier = None
        self.first_day_SubClinical = None 
        self.first_day_Exposed = None 
        self.first_day_Detected = None 
        self.first_day_infected = None 
        self.new_exposures = 0
        self.new_Clinical = 0
        self.new_SubClinical = 0
        self.new_Carrier = 0
        self.new_Detected = 0
        self.npigs = npigs
        self.Clinical = 0
        self.location = location
        self.Susceptible = self.npigs - self.Clinical
        self.Exposed = ninfected
        self.Carrier = 0
        self.SubClinical = 0
        self.deceased = 0
        self.Detected = 0
        self.x, self.y = location
        self.x_min,self.x_max = self.x - size[0]/2, self.x + size[0]/2
        self.y_min,self.y_max = self.y - size[1]/2, self.y + size[1]/2    
        self.pen_id = pen_id 
        self.Q_i = Q_i  
        self.Q_in = Q_in  
        self.V_b = V_b  
        self.first_day_not_all_Susceptible = None
        self.Exposed_delay_counter = [0] * self.npigs
        self.asymp_delay_counter = [0] * self.npigs
        self.symp_delay_counter = [0] * self.npigs
        self.Carrier_delay_counter = [0] * self.npigs
        self.Detected_delay_counter = [0] * self.npigs
    def __repr__(self):
        return f"Pen has {self.npigs} pigs, {self.Clinical} of which are infected."

    def copy(self):
        return copy.deepcopy(self)
    
    def total_population(self):
        return (self.Susceptible + self.Exposed + self.Clinical +
                self.SubClinical + self.Detected + self.Carrier)
        
    def total_populationInf(self):
        return ( self.Clinical + self.SubClinical +  self.Detected )

    def update_Q_i(self, epsilon_1, epsilon_2, Q_shed, current_day):
        self.Q_i = self.Q_i * (1 - epsilon_1) * (1 - epsilon_2) + omega_v * eta_1 * Q_shed * self.total_populationInf() 
    
    def update_Q_in(self, epsilon_1, epsilon_2, Q_shed, current_day):
        self.Q_in = self.Q_in * (1 - epsilon_1) * (1 - epsilon_2) + omega_v * eta_2 * Q_shed * self.total_populationInf()

    def update_V_b(self, epsilon_1, V_b_shed):
        self.V_b = self.V_b  * (1-epsilon_1) + V_b_shed  * self.total_populationInf() 
        
    def update_new_exposures(self, new_cases):
        self.new_exposures = new_cases
        
    def update_new_Clinical(self, new_cases):
        self.new_Clinical = new_cases 
    
    def update_new_SubClinical(self, new_cases):
        self.new_SubClinical = new_cases
            
    def update_new_Carrier(self, new_cases):
        self.new_Carrier = new_cases   
        
    def update_new_Detected(self, new_cases):
        self.new_Detected = new_cases 
class barn:
    def __init__(self, nbarn, nroom, nrow, npen_eachrow, npigs_perpen, n_ini_infected, where_ini_infected, barn_size, pen_size):
        self.nbarn = nbarn
        self.nrow = nrow
        self.nroom = nroom
        self.npen_eachrow = npen_eachrow
        self.npigs_perpen = npigs_perpen
        self.n_ini_infected = n_ini_infected
        self.pen=[]
        self.barn_size=barn_size
        self.pen_size=pen_size
        self.room_width = barn_size[0]/nroom
        self.threshold_crossed = False
        where_ini_infected=np.array(where_ini_infected)
        if len(where_ini_infected.shape)==2:
            code_infected=[]
            for i in range(len(where_ini_infected)):
                code_infected.append(where_ini_infected[i,0]*nroom*nrow*npen_eachrow
                                     +where_ini_infected[i,1]*nrow*npen_eachrow+where_ini_infected[i,2]*npen_eachrow+where_ini_infected[i,3])
        self.where_ini_infected=where_ini_infected
        pen_location=[0,0]
        n_walkways=nroom*int(nrow/2)
        walkway_size=(barn_size[0]-nroom*nrow*pen_size[0])/n_walkways
        self.walkway_size=walkway_size
        self.percentage_contributions_df_all = []
        self.mean_contributions_df_all = []
        self.route_contribution_df_all = []
        self.SE_contributions_df_all = []       
        for barn in range(nbarn):
            for room in range(nroom):
                xrowpos=pen_size[0]/2+barn*(2*self.barn_size[0]+4*self.walkway_size)+room*self.room_width
                for row in range(nrow):
                    if row!=0:
                        xrowpos+=pen_size[0]
                    if row%2==1:
                        xrowpos+=walkway_size
                    for whichpen in range(npen_eachrow):
                        pen_id=(barn,room, row, whichpen)
                        if (len(where_ini_infected.shape)==1 and where_ini_infected.shape[0] != 4) or (len(where_ini_infected.shape)==2 and where_ini_infected.shape[1] != 4):
                            raise ValueError("where_ini_infected must be a list of 4 elements, corresponding to the barn, room, row and pen where the initial infection occurs.")
                        elif len(where_ini_infected.shape)==2:
                            pen_location[0]=xrowpos
                            pen_location[1]=pen_size[1]/2+whichpen*(pen_size[1])
                            code_here_infected=barn*nroom*nrow*npen_eachrow+room*nrow*npen_eachrow+row*npen_eachrow+whichpen
                            if code_here_infected in code_infected:
                                self.pen.append(Pen(npigs_perpen, n_ini_infected, pen_location, pen_size, pen_id))
                            else:
                                self.pen.append(Pen(npigs_perpen, 0, pen_location, pen_size, pen_id))
                        else:
                            pen_location[0]=xrowpos
                            pen_location[1]=pen_size[1]/2+whichpen*(pen_size[1])
                            if np.array_equal((barn, room, row, whichpen),where_ini_infected):
                                self.pen.append(Pen(npigs_perpen, n_ini_infected, pen_location, pen_size, pen_id))
                            else:
                                self.pen.append(Pen(npigs_perpen, 0, pen_location, pen_size, pen_id))
                    
        self.Susceptible=npigs_perpen*nbarn*nroom*nrow*npen_eachrow-n_ini_infected
        self.Exposed,self.SubClinical,self.Detected,self.deceased,self.Carrier = 0,0,0,0,0
        self.Clinical = n_ini_infected
    
    def __repr__(self):
        return f"A model of {self.nbarn} (connected) barn(s), each with {self.nroom} room(s), each with {self.nrow} row(s) of pen(s), each row with {self.npen_eachrow} pen(s), each pen with {self.npigs_perpen} pig(s). Initially, {self.n_ini_infected} pigs are infected with symptoms at (barn,room,row,pen)={self.where_ini_infected} and the remaining are Susceptible."

    def copy(self):
        return copy.deepcopy(self)  
    def update(self, current_day):       
        new_barn=self.copy()        
        i=0
        for a_pen in new_barn.pen:
            new_pen=a_pen.copy()
            new_pen,  SE_contributions_df=transition(a_pen.pen_id,self,current_day)
            new_barn.pen[i]=new_pen 
            if len(new_barn.SE_contributions_df_all)>0:
                self.SE_contributions_df_all = pd.concat([self.SE_contributions_df_all,SE_contributions_df],ignore_index=True, axis=0)
            else:
                self.SE_contributions_df_all = SE_contributions_df
            i+=1
        for pen in self.pen: 
            if pen.Susceptible != pen.npigs and pen.first_day_not_all_Susceptible is None:
                pen.first_day_not_all_Susceptible = current_day
                
        for pen in self.pen:
            if pen.Detected != 0 and pen.first_day_Detected is None: 
                 pen.first_day_Detected = current_day
                 
        for pen in self.pen:
            if (pen.Clinical+pen.SubClinical+pen.Carrier) != 0 and pen.first_day_infected is None:
                 pen.first_day_infected = current_day
        self=new_barn.copy()
        return self
            
    def simulate(self,n_timesteps,current_day):
        for _ in range(n_timesteps):
            self.update(current_day)
        return self
    def simulate_day(barn):
        for pen in barn.pen:
            pen.update_new_exposures(0)
def transition(pen_ID,barn,current_day):
    pen_idx = pen_ID[0] + pen_ID[1] * barn.nrow * barn.npen_eachrow + pen_ID[2] * barn.npen_eachrow + pen_ID[3]
    a_pen = barn.pen[pen_idx]
    ZZ22=a_pen.Q_i
    ZZ222=a_pen.Q_in
    ZZ33=a_pen.V_b
    a_pen.update_Q_i(epsilon_1, epsilon_2, Q_shed, current_day)
    a_pen.update_Q_i(epsilon_1, epsilon_2, Q_shed, current_day)
    a_pen.update_V_b( epsilon_2, V_b_shed)
    a_pen.update_V_b( epsilon_2, V_b_shed)
    
    neighbor_pen_idx = []
    if pen_ID[3] > 0:
        left_neighbor_idx = pen_idx - 1
        neighbor_pen_idx.append(left_neighbor_idx)

    if pen_ID[3] < barn.npen_eachrow - 1:
        right_neighbor_idx = pen_idx + 1
        neighbor_pen_idx.append(right_neighbor_idx)

    pen_idx = pen_ID[0] + pen_ID[1] * barn.nrow * barn.npen_eachrow + pen_ID[2] * barn.npen_eachrow + pen_ID[3]
    all_neighbor_pen_idx = []
    for pen_in_row_idx in range(barn.npen_eachrow):
        if pen_in_row_idx != pen_ID[3]:
           neighbor_idx = pen_ID[0] + pen_ID[1] * barn.nrow * barn.npen_eachrow + pen_ID[2] * barn.npen_eachrow + pen_in_row_idx
           all_neighbor_pen_idx.append(neighbor_idx)

    for row_idx in range(barn.nrow):
        if row_idx != pen_ID[2]:
           for pen_in_row_idx in range(barn.npen_eachrow):
               neighbor_idx = pen_ID[0] + (pen_ID[1] * barn.nrow + row_idx) * barn.npen_eachrow + pen_in_row_idx
               all_neighbor_pen_idx.append(neighbor_idx)

    for barn_idx in range(barn.nbarn):
        if barn_idx != pen_ID[0]:
           for pen_in_row_idx in range(barn.npen_eachrow):
               neighbor_idx = barn_idx * barn.nrow * barn.npen_eachrow + pen_ID[1] * barn.nrow * barn.npen_eachrow + pen_in_row_idx
               all_neighbor_pen_idx.append(neighbor_idx)
    
    prob_within_pen1 = within_pen_contact_direct(pen_idx,beta_dwp)
    prob_within_pen2 = within_pen_contact_fecal(pen_idx, beta_fwp, a_pen.Q_i, Q_j)
    prob_within_pen3 = within_pen_contact_air(pen_idx,a_pen.V_b,beta_awp)
    prob_bet1 = np.sum(between_pen_contact1(pen_idx, neighbor_pen_idx, beta_fbp))
    prob_bet2 = np.sum(between_pen_contact2(pen_idx, neighbor_pen_idx, beta_fwp,a_pen.Q_in,Q_j))
    prob_room = np.array(room_transmission( beta_awp))
    prob_room_copy = prob_room.copy()
    for i in range(len(prob_room_copy)):
        if i != (pen_idx ):  
            prob_room_copy[i] = 0
    prob_room1 = np.sum(prob_room_copy)
    prob_human_pen =human_contact(pen_idx,C_hum) 
    ZZ = (barn.pen[pen_idx].Susceptible + barn.pen[pen_idx].Exposed + barn.pen[pen_idx].Clinical + barn.pen[pen_idx].SubClinical +
           barn.pen[pen_idx].Detected + barn.pen[pen_idx].Carrier)
    ZZ1 = barn.pen[pen_idx].Clinical+barn.pen[pen_idx].SubClinical + barn.pen[pen_idx].Detected
    if ZZ == 0:
       ZZ2=0
    else:
       ZZ2=ZZ22
    daily_contributions = {"Within1": 0, "Within2": 0,"Within3": 0,"Bet1":0,"Bet2":0,"Room":0,"Human":0}
    
    if ZZ & ZZ1 == 0:
       contribution_within_pen1 = 0  
       contribution_within_pen2 = 0
       contribution_within_pen3 = 0
    else:
       contribution_within_pen1 = prob_within_pen1 * (ZZ1 / ZZ)
       contribution_within_pen2 = ZZ2*prob_within_pen2 * (ZZ1/ ZZ)
       contribution_within_pen3 = ZZ33*prob_within_pen3 * (ZZ1 / ZZ)
    total_neighbor_population = 0
    neighbor_ZZ1 = 0  
    for neighbor_idx in neighbor_pen_idx:
        neighbor_population = (barn.pen[neighbor_idx].Susceptible + barn.pen[neighbor_idx].Exposed +
                            barn.pen[neighbor_idx].Clinical + barn.pen[neighbor_idx].SubClinical +
                             barn.pen[neighbor_idx].Detected +barn.pen[neighbor_idx].Carrier)
        neighbor_infected = barn.pen[neighbor_idx].Clinical + barn.pen[neighbor_idx].SubClinical + barn.pen[neighbor_idx].Detected
        total_neighbor_population += neighbor_population
        neighbor_ZZ1 += neighbor_infected

    if neighbor_infected or neighbor_population == 0:
        ZZ4 = 0
    else:
        ZZ4 = ZZ222
    if neighbor_ZZ1 ==0 or total_neighbor_population == 0:
        contribution_between_pen1 = 0
        contribution_between_pen2 = 0
    else:
        contribution_between_pen1 = prob_bet1 * (neighbor_ZZ1/ (total_neighbor_population+NN))
        contribution_between_pen2 = ZZ4 * prob_bet2 * (neighbor_ZZ1/ (total_neighbor_population+NN))
    total_network_population = 0
    total_network_infected = 0  

    for neighbor_idx in all_neighbor_pen_idx:
        neighbor_population_network = (barn.pen[neighbor_idx].Susceptible + barn.pen[neighbor_idx].Exposed +
                            barn.pen[neighbor_idx].Clinical + barn.pen[neighbor_idx].SubClinical +
                             barn.pen[neighbor_idx].Detected + barn.pen[neighbor_idx].Carrier)
        neighbor_infected_network = (barn.pen[neighbor_idx].Clinical + barn.pen[neighbor_idx].SubClinical + 
                                    barn.pen[neighbor_idx].Detected)
        total_network_population += neighbor_population_network
        total_network_infected += neighbor_infected_network

    current_pen_population = (barn.pen[pen_idx].Susceptible + barn.pen[pen_idx].Exposed + barn.pen[pen_idx].Clinical + 
                            barn.pen[pen_idx].SubClinical + barn.pen[pen_idx].Detected + barn.pen[pen_idx].Carrier)
    current_pen_infected = (barn.pen[pen_idx].Clinical + barn.pen[pen_idx].SubClinical + barn.pen[pen_idx].Detected)

    total_network_population += current_pen_population
    total_network_infected += current_pen_infected

    if total_network_infected == 0 or total_network_population == 0:
        contribution_room = 0
        contribution_human = 0
    else:
        contribution_room = ZZ33 * prob_room1 * (neighbor_infected_network / (neighbor_population_network+NN))
        contribution_human = prob_human_pen * (total_network_infected / (total_network_population+NN))

    daily_contributions["Within1"] = contribution_within_pen1
    daily_contributions["Within2"] = contribution_within_pen2
    daily_contributions["Within3"] = contribution_within_pen3
    daily_contributions["Bet1"] = contribution_between_pen1
    daily_contributions["Bet2"] = contribution_between_pen2
    daily_contributions["Room"] = contribution_room
    daily_contributions["Human"] = contribution_human
    prob_inf = 1 - math.exp(-contribution_within_pen1-contribution_within_pen2-contribution_within_pen3-contribution_between_pen1-
                            contribution_between_pen2-contribution_room-contribution_human)
    p1 = 0.15
    q = 0.8
    r = 0.05
    average_incubation_days = np.random.gamma(13.299,0.3384482)  
    Latency = 1 / average_incubation_days
    prob_Exposed_asymp = Latency * p1
    prob_Exposed_symp = Latency * q
    prob_Exposed_scarr = Latency * r
    prob_mortality_Exposed = (1 - (1 - p) ** (1 / average_incubation_days) )*MortalityH 
    prob_stay_Exposed = max(0, 1 - prob_Exposed_asymp - prob_Exposed_symp - prob_Exposed_scarr - prob_mortality_Exposed)

    total_prob = prob_Exposed_asymp + prob_Exposed_symp + prob_Exposed_scarr + prob_stay_Exposed + prob_mortality_Exposed
    prob_Exposed_asymp /= total_prob
    prob_Exposed_symp /= total_prob
    prob_Exposed_scarr /= total_prob
    prob_stay_Exposed /= total_prob
    prob_mortality_Exposed /= total_prob

    num_Exposed = a_pen.Exposed  
    Exposed_transition = []
    if current_day >= average_incubation_days:
        Exposed_transition = np.random.choice(
            ['asymp', 'symp', 'Exposed', 'carr', 'dece'],
            size=num_Exposed,
            p=[prob_Exposed_asymp, prob_Exposed_symp, prob_stay_Exposed, prob_Exposed_scarr, prob_mortality_Exposed]
        ).tolist()

    average_recovery_days = np.random.uniform(14,44)
    prob_asymp_rec = 1 / average_recovery_days
    prob_asymp_symp = np.random.uniform(0, 0.1)
    prob_mortality_asymp = MortalityH*1/14
    prob_asymp_det = 0.01
    prob_stay_asymp = 1 - prob_asymp_rec - prob_asymp_symp - prob_asymp_det - prob_mortality_asymp

    if current_day >= average_recovery_days:
        asymp_transition = np.random.choice(
            ['rec', 'symp', 'asymp', 'adet', 'deca'], 
            size=barn.pen[pen_idx].SubClinical, 
            p=[prob_asymp_rec, prob_asymp_symp, prob_stay_asymp, prob_asymp_det, prob_mortality_asymp]
        ).tolist()
    else:
        asymp_transition = ['asymp'] * barn.pen[pen_idx].SubClinical

    prob_symp_rec = 1 / average_recovery_days
    prob_symp_det = 0.01
    prob_symp_asymp = np.random.uniform(0, 0.01)
    prob_mortality_symp = MortalityH * 1/8.3
    prob_stay_symp = 1 - prob_symp_rec - prob_symp_det - prob_symp_asymp - prob_mortality_symp

    if current_day >= average_recovery_days:
        symp_transition = np.random.choice(
            ['rec', 'det', 'asymp', 'symp', 'decs'], 
            size=barn.pen[pen_idx].Clinical, 
            p=[prob_symp_rec, prob_symp_det, prob_symp_asymp, prob_stay_symp, prob_mortality_symp]
        ).tolist()
    else:
        symp_transition = ['symp'] * barn.pen[pen_idx].Clinical

    prob_carr_asymp = np.random.uniform(0, 0.01)
    prob_carr_symp = np.random.uniform(0, 0.01)
    prob_carr_det = 0.01
    prob_carr_rec = 1 / average_recovery_days
    prob_mortality_carr = (1 - (1 - p) ** (1 / average_recovery_days) )*MortalityH 
    prob_carr_stay = 1 - prob_carr_asymp - prob_carr_symp - prob_carr_det - prob_carr_rec - prob_mortality_carr

    if current_day >= average_recovery_days:
        carr_transition = np.random.choice(
            ['recc', 'sympc', 'asympc', 'detc', 'decc', 'stay'], 
            size=barn.pen[pen_idx].Carrier,  
            p=[prob_carr_rec, prob_carr_symp, prob_carr_asymp, prob_carr_det, prob_mortality_carr, prob_carr_stay]
        ).tolist()
    else:
        carr_transition = ['stay'] * barn.pen[pen_idx].Carrier

    prob_mortality_det = 0.06*MortalityH
    prob_stay_Detected = 1 - prob_mortality_det

    Detected_transition = np.random.choice(['decd', 'dstay'], 
                                       size=barn.pen[pen_idx].Detected, p=[prob_mortality_det, prob_stay_Detected]).tolist()
    prob_mortality_sus = 1 - (1 - p) ** (1 / T)                                     
    S_to_E = 0
    deceased_from_Susceptible = 0
    
    route_contribution = {"Within1": 0, "Within2": 0, "Within3": 0, "Bet1": 0, "Bet2": 0, "Room": 0, "Human": 0}
    daily_contributions_data = []
    if barn.pen[pen_idx].Susceptible > 0:
        for _ in range(barn.pen[pen_idx].Susceptible):
            random_prob = np.random.rand()
            if random_prob < contribution_within_pen1:
                S_to_E += 1
                route_contribution["Within1"] += 1
            elif random_prob < contribution_within_pen1 + contribution_within_pen2:
                S_to_E += 1
                route_contribution["Within2"] += 1
            elif random_prob < contribution_within_pen1 + contribution_within_pen2 + contribution_within_pen3:
                S_to_E += 1
                route_contribution["Within3"] += 1
            elif random_prob < contribution_within_pen1 + contribution_within_pen2 + contribution_within_pen3 + contribution_between_pen1:
                S_to_E += 1
                route_contribution["Bet1"] += 1
            elif random_prob < contribution_within_pen1 + contribution_within_pen2 + contribution_within_pen3 + contribution_between_pen1 + contribution_between_pen2:
                S_to_E += 1
                route_contribution["Bet2"] += 1
            elif random_prob < contribution_within_pen1 + contribution_within_pen2 + contribution_within_pen3 + contribution_between_pen1 + contribution_between_pen2 + contribution_room:
                S_to_E += 1
                route_contribution["Room"] += 1
            elif random_prob < prob_inf:  
                S_to_E += 1
                route_contribution["Human"] += 1
            elif random_prob < prob_inf + prob_mortality_sus:
                deceased_from_Susceptible += 1            
    total_transitions = sum(route_contribution.values())
    Dpercentage_contributions = {route: (count / total_transitions) * 100 if total_transitions > 0 else 0 
                                    for route, count in route_contribution.items()}
    columns = ['Day', 'Within1', 'Within2', 'Within3', 'Bet1', 'Bet2', 'Room', 'Human']
    Dcontributions_df = pd.DataFrame(daily_contributions_data, columns=columns)

    current_folder = os.getcwd()
    output_file_path = os.path.join(current_folder, "S_to_E_contributions_percentage.csv")

    Dcontributions_df.to_csv(output_file_path, index=False)    
    SE_contributions_df = pd.DataFrame(daily_contributions_data) 

    barn.pen[pen_idx].Susceptible -= S_to_E + deceased_from_Susceptible
    barn.pen[pen_idx].Exposed += S_to_E
    barn.pen[pen_idx].Exposed -= (Exposed_transition.count('asymp') + Exposed_transition.count('symp')+
                                  Exposed_transition.count('carr')+Exposed_transition.count('dece'))
    barn.pen[pen_idx].SubClinical += Exposed_transition.count('asymp') + symp_transition.count('asymp')+carr_transition.count('asympc')
    barn.pen[pen_idx].SubClinical -= ( asymp_transition.count('symp')+asymp_transition.count('adet')+
                                      asymp_transition.count('deca'))
    barn.pen[pen_idx].Clinical += (Exposed_transition.count('symp') + asymp_transition.count('symp')+carr_transition.count('sympc'))
    barn.pen[pen_idx].Clinical -= (symp_transition.count('asymp') +symp_transition.count('sdet')
                                   +symp_transition.count('decs'))
    barn.pen[pen_idx].Carrier += Exposed_transition.count('carr') +Exposed_transition.count('stay')
    barn.pen[pen_idx].Carrier -= ( carr_transition.count('sympc') + carr_transition.count('asympc') 
                                 +carr_transition.count('detc') )+carr_transition.count('decc')
    barn.pen[pen_idx].Detected += carr_transition.count('detc') +asymp_transition.count('adet')+symp_transition.count('sdet')
    barn.pen[pen_idx].Detected -= Detected_transition.count('decd')
    barn.pen[pen_idx].deceased += (Exposed_transition.count('dece')+asymp_transition.count('deca')+symp_transition.count('decs')+
                                  carr_transition.count('decc')+Detected_transition.count('decd'))+deceased_from_Susceptible
    prob_detection_low = 0.01  
    prob_detection_high_Clinical = 0.95  
    prob_detection_high_SubClinical = 0.75  
    prob_detection_high_Carrier = 0.5  

    if not barn.threshold_crossed:
        total_population = sum([pen.Susceptible + pen.Exposed + pen.SubClinical + pen.Clinical + 
                                 pen.Carrier + pen.Detected for pen in barn.pen])
        total_deceased = sum([pen.deceased for pen in barn.pen])
        percent_deceased = (total_deceased / total_population) * 100
        if percent_deceased >= 10:
            barn.threshold_crossed = True

    if barn.threshold_crossed:
        detection_probabilityC = prob_detection_high_Clinical
        detection_probabilitySC = prob_detection_high_SubClinical
        detection_probabilityCC = prob_detection_high_Carrier
    else:
        detection_probabilityC = prob_detection_low
        detection_probabilitySC = prob_detection_low
        detection_probabilityCC = prob_detection_low

    detection_efficiency = 0.9  
    Detected_Clinical = int(barn.pen[pen_idx].Clinical * detection_probabilityC * detection_efficiency)
    Detected_SubClinical = int(barn.pen[pen_idx].SubClinical * detection_probabilitySC * detection_efficiency)
    Detected_Carrier = int(barn.pen[pen_idx].Carrier * detection_probabilityCC * detection_efficiency)

    barn.pen[pen_idx].Clinical -= Detected_Clinical
    barn.pen[pen_idx].SubClinical -= Detected_SubClinical
    barn.pen[pen_idx].Carrier -= Detected_Carrier
    barn.pen[pen_idx].Detected += Detected_Clinical + Detected_SubClinical + Detected_Carrier    

    num_new_Exposed =  S_to_E
    if num_new_Exposed > 0 and barn.pen[pen_idx].first_day_Exposed is None:
       barn.pen[pen_idx].first_day_Exposed = current_day
       
    barn.pen[pen_idx].update_new_exposures(S_to_E)   
    barn.pen[pen_idx].update_new_Clinical((Exposed_transition.count('symp') + asymp_transition.count('symp')+carr_transition.count('sympc')))
    barn.pen[pen_idx].update_new_SubClinical(Exposed_transition.count('asymp') + symp_transition.count('asymp')+carr_transition.count('asympc'))
    barn.pen[pen_idx].update_new_Carrier(Exposed_transition.count('carr') +Exposed_transition.count('stay'))
    barn.pen[pen_idx].update_new_Detected(carr_transition.count('detc') +asymp_transition.count('adet')+symp_transition.count('sdet')+Detected_Clinical + Detected_SubClinical + Detected_Carrier )
        
    return Pen,  SE_contributions_df

def within_pen_contact_direct(pen_idx,beta_dwp):
    prob_within_pen_infection1 = beta_dwp
    return prob_within_pen_infection1

def within_pen_contact_fecal(pen_idx, beta_fwp, Q_i, Q_j):
    prob_within_pen_infection2 = beta_fwp * Q_j * xi_1
    return prob_within_pen_infection2

def within_pen_contact_air(pen_idx,V_b,beta_awp):
    prob_within_pen_infection3 = beta_awp * 25.875
    return prob_within_pen_infection3

def between_pen_contact1(pen_idx, neighbor_pen_idx, beta_fbp):
    Prob_BetweenPenInfection = beta_fbp*0.5
    result_matrix = np.zeros(len(neighbor_pen_idx))
    for i, neighbor_idx in enumerate(neighbor_pen_idx):
        result_matrix[i] = Prob_BetweenPenInfection
    return result_matrix   

def between_pen_contact2(pen_idx, neighbor_pen_idx, beta_fwp,Q_in,Q_j):
    Prob_BetweenPenInfection1 = beta_fwp * Q_j * xi_2 * 0.5
    result_matrix1 = np.zeros(len(neighbor_pen_idx))
    for i, neighbor_idx in enumerate(neighbor_pen_idx):
        result_matrix1[i] = Prob_BetweenPenInfection1
    return result_matrix1  

import os
import pandas as pd
import numpy as np

def room_transmission(beta_awp):
    try:
        df = pd.read_csv(file_path)
        row_values = df.iloc[:, 0] * beta_awp * 25.875
        final_result = row_values.values.reshape(-1, 1)
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        final_result = np.array([])   
    return final_result

# Example loop to handle multiple room files
data_folder = '/Users/adeka/Documents/Workspace/NetworkPaper/RoomTrials/BetweenRoomTransmission'  
room_files = [f for f in os.listdir(data_folder) if f.endswith('.csv') and f.startswith('00')]

for room_file in room_files:
    # Construct the full file path for each room
    room_file_path = os.path.join(data_folder, room_file)

    
    # Call the room_transmission function with the room-specific file
    room_transmission_data = room_transmission(beta_awp)
    
    # Continue processing the room-specific transmission data in your simulation
    print(f"Processed transmission data for {room_file}")


def human_contact(pen_idx,C_hum):
    prob_humanInf = C_hum
    return prob_humanInf


import random
def transfer_pigs(barn, num_transfers=3, num_source_pens=3, num_target_pens=3):
    transfer_probability =0.05
    total_transferred_pigs = 0  

    if random.random() < transfer_probability:
        source_pen_indices = random.sample(range(len(barn.pen)), num_transfers * num_source_pens)
        target_pen_indices = random.sample(range(len(barn.pen)), num_transfers * num_target_pens)
        mean_pigs_to_transfer = 1
        num_pigs_to_transfer = np.random.poisson(mean_pigs_to_transfer, num_transfers)

        compartments = ['Susceptible', 'Exposed', 'SubClinical', 'Clinical', 'Carrier', 'Detected']

        for transfer_index in range(num_transfers):
            transferred_compartments = []
            source_pen_idx = None

            for _ in range(num_pigs_to_transfer[transfer_index]):
                for idx in range(num_source_pens):
                    temp_source_pen_idx = source_pen_indices[transfer_index * num_source_pens + idx]
                    for comp in compartments:
                        if getattr(barn.pen[temp_source_pen_idx], comp) >= 1:
                            source_pen_idx = temp_source_pen_idx
                            setattr(barn.pen[source_pen_idx], comp, getattr(barn.pen[source_pen_idx], comp) - 1)
                            total_transferred_pigs += 1  
                            transferred_compartments.append(comp)
                            break
                    if source_pen_idx is not None:
                        break
            if source_pen_idx is not None:
                target_pen_idx = target_pen_indices[transfer_index * num_target_pens]
                for comp in transferred_compartments:
                    setattr(barn.pen[target_pen_idx], comp, getattr(barn.pen[target_pen_idx], comp) + 1)
    return total_transferred_pigs
import copy
import random
barn_size = (20.5,140)
pen_size = (10, 10)
num_days = 30
n_timesteps = 1
NSim= 1
nbarn=1
data_folder = '/Users/adeka/Documents/Workspace/NetworkPaper/RoomTrials/BetweenRoomTransmission'


for room_file in room_files:
    file_path = os.path.join(data_folder, room_file)
    data = pd.read_csv(file_path, header=None)
    total_row_count = data.shape[0]
    non_na_row_count = data.iloc[:, 0].dropna().shape[0]
    npen_eachrow = non_na_row_count // 2
    print(f'Room file: {room_file}')
    print(f'Total rows: {total_row_count}, Non-NA rows in the first column: {non_na_row_count}')
    print(f'The number of pens in each row is: {npen_eachrow}')
    
nroom=1
nrow=2
all_total_Susceptible_results = []
all_total_Exposed_results = []
all_total_Clinical_results = []
all_total_SubClinical_results = []
all_total_Carrier_results = []
all_total_Detected_results = []
all_total_deceased_results = []

current_folder = os.getcwd()
output_directory = os.path.join(current_folder)
csv_path = '/Users/adeka/Documents/Workspace/May/PathwaysandSIR2T/barns_with_pen_capacity.csv'
barn_df = pd.read_csv(csv_path) 
from itertools import product
all_combinations = list(product(range(nbarn), range(nroom), range(nrow), range(npen_eachrow)))
data_folder ='/Users/adeka/Documents/Workspace/NetworkPaper/RoomTrials/BetweenRoomTransmission'
barn_states = []

for simulation_number in range(1, NSim + 1):
    daily_not_all_Susceptible_count = [] 
    daily_Detected_count = []
    daily_infected_count = []
    daily_infectednotDetected_count = []
    where_ini_infected = random.choice(all_combinations)
    for room_file in room_files:
        barn_id = os.path.splitext(room_file)[0][:9]  
    
        npigs_perpen = None
        if barn_id in barn_df['barn_id'].str[:9].values:
            npigs_perpen = barn_df[barn_df['barn_id'].str[:9] == barn_id]['per_pigs_pen'].iloc[0]
        else:
            print(f"barn_id {barn_id} not found in CSV.")
            continue
        if npigs_perpen is not None:
            mybarn = barn(
                nbarn=1,nroom=1, nrow=2, npen_eachrow=npen_eachrow, npigs_perpen=npigs_perpen,n_ini_infected=1, where_ini_infected=where_ini_infected,barn_size=barn_size, pen_size=pen_size
            )
            print(f"Barn initialized for barn_id {barn_id} with npigs_perpen: {npigs_perpen}")
            file_path = os.path.join(data_folder, room_file)
            room_data = pd.read_csv(file_path)
            mybarn.simulate(n_timesteps, simulation_number)
            barn_states.append(mybarn) 
        else:
            print(f"No valid npigs_perpen found for barn_id {barn_id}.")

    print(f"Simulation {simulation_number} completed.")

    total_Susceptible_results = [npigs_perpen *npen_eachrow*2]
    total_Exposed_results = [1]
    total_Clinical_results = [0]
    total_SubClinical_results = [0]
    total_Carrier_results = [0]
    total_Detected_results = [0]
    total_deceased_results = [0]
    
    total_V_b_results = [0]
    total_Q_i_results = [0]

    total_new_exposures_results = [0]
    total_new_Clinical_results = [0]
    total_new_SubClinical_results = [0]
    total_new_Carrier_results = [0]
    total_new_exposure_results = [0]
    total_new_Detected_results = [0]
    daily_not_all_Susceptible_count = [0]
    daily_Detected_count = [0]
    daily_infected_count = [0]
    daily_infectednotDetected_count = [0]
    
    current_simulation_data = []
    for day in range(1, num_days + 1):    
        mybarn.simulate(n_timesteps, day)
                
        count_not_all_Susceptible = sum(1 for pen in mybarn.pen if pen.Susceptible != pen.npigs)
        daily_not_all_Susceptible_count.append(count_not_all_Susceptible)
        
        count_Detected = sum(1 for pen in mybarn.pen if pen.Detected != 0)
        daily_Detected_count.append(count_Detected)
        
        count_infected = sum(1 for pen in mybarn.pen if (pen.Clinical + pen.SubClinical + pen.Carrier) != 0 )
        daily_infected_count.append(count_infected)
        
        count_infected_notDetected = sum(1 for pen in mybarn.pen if (pen.Clinical + pen.SubClinical + pen.Carrier) != 0 and pen.Detected == 0)
        daily_infectednotDetected_count.append(count_infected_notDetected)

        total_Susceptible_results.append(sum([b_pen.Susceptible for b_pen in mybarn.pen]))
        total_Exposed_results.append(sum([b_pen.Exposed for b_pen in mybarn.pen]))
        total_Clinical_results.append(sum([b_pen.Clinical for b_pen in mybarn.pen]))
        total_SubClinical_results.append(sum([b_pen.SubClinical for b_pen in mybarn.pen]))
        total_Carrier_results.append(sum([b_pen.Carrier for b_pen in mybarn.pen]))
        total_Detected_results.append(sum([b_pen.Detected for b_pen in mybarn.pen]))
        total_deceased_results.append(sum([b_pen.deceased for b_pen in mybarn.pen]))
        
        daily_new_exposures = sum(pen.new_exposures for pen in mybarn.pen)
        total_new_exposures_results.append((daily_new_exposures/(npigs_perpen*npen_eachrow*2))*100)
        
        daily_new_Clinical = sum(pen.new_Clinical for pen in mybarn.pen)
        total_new_Clinical_results.append((daily_new_Clinical/(npigs_perpen*npen_eachrow*2))*100)
        
        daily_new_SubClinical = sum(pen.new_SubClinical for pen in mybarn.pen)
        total_new_SubClinical_results.append((daily_new_SubClinical/(npigs_perpen*npen_eachrow*2))*100)
        
        daily_new_Carrier = sum(pen.new_Carrier for pen in mybarn.pen)
        total_new_Carrier_results.append((daily_new_Carrier/(npigs_perpen*npen_eachrow*2))*100)
        
        daily_new_Detected = sum(pen.new_Detected for pen in mybarn.pen)
        total_new_Detected_results.append((daily_new_Detected/(npigs_perpen*npen_eachrow*2))*100)

    # Collect data for each pen in this simulation and day
        for pen_idx, pen in enumerate(mybarn.pen):
            pen_data = {
                'Simulation': simulation_number,
                'Day': day,
                'Pen ID': pen_idx,
                'Susceptible': pen.Susceptible,
                'Exposed': pen.Exposed,
                'Clinical': pen.Clinical,
                'SubClinical': pen.SubClinical,
                'Carrier': pen.Carrier,
                'Detected': pen.Detected,
                'Deceased': pen.deceased,
                'New Exposures': pen.new_exposures,
                'New Clinical': pen.new_Clinical,
                'New SubClinical': pen.new_SubClinical,
                'New Carrier': pen.new_Carrier,
                'New Detected': pen.new_Detected
            }
            current_simulation_data.append(pen_data)
        simulation_df = pd.DataFrame(current_simulation_data)
        filename = f'combinedSIR_{barn_id}_simulation_{simulation_number}.csv'
        simulation_df.to_csv(os.path.join(data_folder, filename), index=False)
        print(f"Saved simulation {simulation_number} for barn_id {barn_id} to {filename}")

    df_all_data = pd.DataFrame({
        'Day': range(0, num_days + 1),
        'New Exposures': total_new_exposures_results,
        'Cumulative Exposures': pd.Series(total_new_exposures_results).cumsum(),
        'New Clinical': total_new_Clinical_results,
        'Cumulative Clinical': pd.Series(total_new_Clinical_results).cumsum(),
        'New SubClinical': total_new_SubClinical_results,
        'Cumulative SubClinical': pd.Series(total_new_SubClinical_results).cumsum(),
        'New Carrier': total_new_Carrier_results,
        'Cumulative Carrier': pd.Series(total_new_Carrier_results).cumsum(),
        'New Detected': total_new_Detected_results,
        'Cumulative Detected': pd.Series(total_new_Detected_results).cumsum()
    })

    combined_data_csv_file = os.path.join(output_directory, f'combined_data_simulation_{simulation_number}.csv')
    df_all_data.to_csv(combined_data_csv_file, index=False)
    print(f"All simulation data saved to {combined_data_csv_file}")

    all_total_Susceptible_results.append(total_Susceptible_results)
    all_total_Exposed_results.append(total_Exposed_results)
    all_total_Clinical_results.append(total_Clinical_results)
    all_total_SubClinical_results.append(total_SubClinical_results)
    all_total_Carrier_results.append(total_Carrier_results)
    all_total_Detected_results.append(total_Detected_results)
    all_total_deceased_results.append(total_deceased_results)
    
    def convert_to_percentage(nested_list):
        return [[value/(npen_eachrow*nrow*npigs_perpen) * 100 for value in inner_list] for inner_list in nested_list]

    all_total_Susceptible_results = convert_to_percentage(all_total_Susceptible_results)
    all_total_Exposed_results = convert_to_percentage(all_total_Exposed_results)
    all_total_Clinical_results = convert_to_percentage(all_total_Clinical_results)
    all_total_SubClinical_results = convert_to_percentage(all_total_SubClinical_results)
    all_total_Carrier_results = convert_to_percentage(all_total_Carrier_results)
    all_total_Detected_results = convert_to_percentage(all_total_Detected_results)
    all_total_deceased_results = convert_to_percentage(all_total_deceased_results)

    dataFraction = {
    'Susceptible': all_total_Susceptible_results[0],
    'Exposed': all_total_Exposed_results[0],
    'Clinical': all_total_Clinical_results[0],
    'SubClinical': all_total_SubClinical_results[0],
    'Carrier': all_total_Carrier_results[0],
    'Detected': all_total_Detected_results[0],
    'Deceased': all_total_deceased_results[0]
     }
    df = pd.DataFrame(dataFraction)
    mean_data_df = pd.DataFrame({
        'Day': range(0, num_days + 1),
        ' Susceptible': total_Susceptible_results,
        ' Exposed': total_Exposed_results,
        ' Clinical': total_Clinical_results,
        ' SubClinical': total_SubClinical_results,
        ' Carrier': total_Carrier_results,
        ' Detected': total_Detected_results,
        ' Deceased': total_deceased_results
    })

Processed transmission data for 00C2G6N_2_1.csv
Processed transmission data for 00BSBWB_1_1.csv
Processed transmission data for 00AZKX3_11_1.csv
Processed transmission data for 00C2NNX_2_1.csv
Processed transmission data for 00C50Y5_1_1.csv
Processed transmission data for 00C3THD_12_1.csv
Processed transmission data for 00CACS3_7_1.csv
Processed transmission data for 00CAEL9_4_1.csv
Processed transmission data for 00AZKU9_7_1.csv
Processed transmission data for 00AZZ60_1_1.csv
Room file: 00C2G6N_2_1.csv
Total rows: 46, Non-NA rows in the first column: 46
The number of pens in each row is: 23
Room file: 00BSBWB_1_1.csv
Total rows: 24, Non-NA rows in the first column: 24
The number of pens in each row is: 12
Room file: 00AZKX3_11_1.csv
Total rows: 42, Non-NA rows in the first column: 42
The number of pens in each row is: 21
Room file: 00C2NNX_2_1.csv
Total rows: 42, Non-NA rows in the first column: 42
The number of pens in each row is: 21
Room file: 00C50Y5_1_1.csv
Total rows: 42, Non-NA